In [1]:
# Import statements
import numpy as np
import random as rand
import os
import pandas
import time

In [2]:
# This cell contains all Connect4 related functions.

# This is a wrapper class that holds the logic engine and returns a decision.
class logic_engine_wrapper:

    # Defaults logic_engine to 0 so that it will just move over one space each turn.
    def __init__(self,logic_engine=0):
        self.logic_engine = logic_engine

    # Returns the logic_engine's move based on the provided board.
    def get_move(self,board):
        if type(self.logic_engine) is int:
            if self.logic_engine >= 6:
                self.logic_engine = 0
                return 6
            else:
                self.logic_engine += 1
                return self.logic_engine - 1
        else:
            output = self.logic_engine.get_output_vector(board.get_board_vector())
            greatest = 0
            for i in range(len(output)):
                if output[i] > output[greatest]:
                    greatest = i
            return greatest
                    
# This class represents a Connect4 board and contains all necessary utilities for it.
# On boards player1 = 1, player2 = -1, and blank = 0.
# The higher the first index the higher on the board i.e. index [5][6] is the top right corner of the board.
class game_board:

    # Creates a 7 x 6 game board.
    def __init__(self):
        self.board = [[0.0 for _ in range(7)] for _ in range(6)]

    # Prints the current game board.
    def print_board(self,player1_symbol='@',player2_symbol='#',clear=False):
        if clear:
            os.system('cls')
        for row in self.board[::-1]:
            print('|',end='')
            for num in row:
                symbol = player1_symbol if num == 1 else (player2_symbol if num == -1 else ' ')
                print(symbol,end='')
            print('|')
        print('---------')
        print(' 1234567 ')

    # Drops a piece into the indicated slot using a range of [0,6]
    def drop_piece(self,slot,player):
        if slot > 6 or slot < 0:
            return False
        else:
            for i in range(6):
                if self.board[i][slot] == 0:
                    self.board[i][slot] = player
                    return True
            return False

    # Returns the board squashed into a numpy array. (This is a utility function for net use)
    def get_board_vector(self):
        board_vector = []
        for row in self.board:
            board_vector += row
        return np.array(board_vector)

    # Returns a player if one of them as won otherwise returns 0.
    def get_winner(self):
        # Checks rows.
        for row in self.board:
            previous = None
            count = 0
            for num in row:
                if num == previous:
                    count += 1
                else:
                    previous = num
                    count = 1
                if count >= 4 and previous != 0:
                    return previous
        # Checks columns.
        for x in range(7):
            previous = None
            count = 0
            for y in range(6):
                num = self.board[y][x]
                if num == previous:
                    count += 1
                else:
                    previous = num
                    count = 1
                if count >= 4 and previous != 0:
                    return previous
        # Checks diagonals.
        for y in range(6):
            for x in range(7):
                num = self.board[y][x]
                if num != 0 and y < 3:
                    if x < 4:
                        if num == self.board[y+1][x+1] and num == self.board[y+2][x+2] and num == self.board[y+3][x+3]:
                            return num
                    elif x > 2:
                        if num == self.board[y+1][x-1] and num == self.board[y+2][x-2] and num == self.board[y+3][x-3]:
                            return num

        return 0


# This function enables the user to play against a logic_engine or lets two logic_engines play against each other.
def play_game(logic_engines=[logic_engine_wrapper()],verbose=True,ai_delay=0.2):
    board = game_board()
    current_player = 1 if rand.choice([True,False]) else -1
    max_num_of_turns = 6 * 7
    for turn in range(max_num_of_turns):
        if verbose:
            board.print_board(clear=True)
        if len(logic_engines) == 2:
            if verbose:
                print("Thinking...")
            time.sleep(ai_delay)
            slot = None
            if current_player == 1:
                proxy_board = game_board()
                for y in range(len(proxy_board.board)):
                    for x in range(len(proxy_board.board[y])):
                        proxy_board.board[y][x] = -1 * board.board[y][x]
                slot = logic_engines[0].get_move(proxy_board)
            else:
                slot = logic_engines[1].get_move(board)
            board.drop_piece(slot,current_player)
        else:
            if current_player == 1:
                slot = int(input('Input a slot: ')) - 1
                board.drop_piece(slot,current_player)
            else:
                if verbose:
                    print("Thinking...")
                time.sleep(ai_delay)
                slot = logic_engines[0].get_move(board)
                board.drop_piece(slot,current_player)
        winner = board.get_winner()
        if winner != 0:
            if verbose:
                board.print_board(clear=True)
            return winner
        else:
            current_player = 1 if current_player == -1 else -1
        if turn == (6 * 7) - 1 and verbose:
            board.print_board(clear=True)
    return 0

In [3]:
#Non-linear functions are declared in this cell.

#Sigmoid function.
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [4]:
# Info String Seperators:
net_section_seperator = '|'
net_sub_section_seperator = ':' # i.e. each hidden layer and each bias layer
net_matrix_row_seperator = '>'
net_vector_index_seperator = ','

In [5]:
# Feedforward network class that's used.
class net:
    
    # wiki_net class constructor.
    def __init__(self,input_size,hidden_layers,non_linears=sigmoid,use_biases=True,name="N/A"):
        self.hidden_layers = []
        self.non_linears = []
        self.biases = []
        self.name = name
        if type(non_linears) != list:
            non_linears = [non_linears] * len(hidden_layers) 
            previous_size = input_size
        elif len(non_linears) == 1:
            non_linears *= len(hidden_layers)
        for i in range(len(hidden_layers)):
            self.hidden_layers.append(2 * np.random.random((previous_size,hidden_layers[i])) - 1)
            self.non_linears.append(non_linears[i])
            self.biases.append(2 * np.random.random(hidden_layers[i]) - 1 if use_biases else np.zeros(hidden_layers[i]))
            previous_size = hidden_layers[i]
    
    # Gets the output vector for a given input vector.
    def get_output_vector(self,input_vector):
        output_vector = input_vector
        for i in range(len(self.hidden_layers)):
            output_vector = self.non_linears[i](np.dot(output_vector,self.hidden_layers[i] + self.biases[i]))
        return output_vector
    
    # Returns a string that contains all of the network's features in the following order, [Name, Score, Hidden_Layers, Biases]
    def generate_info_string(self,score=0):
        info_string = f'{self.name}{net_section_seperator}{score}{net_section_seperator}'
        count_1 = 0
        count_2 = 0
        count_3 = 0
        for layer in self.hidden_layers:
            for vector in layer:
                for index in vector:
                    info_string += f'{index}'
                    info_string += net_vector_index_seperator if count_1 != len(vector) - 1 else ''
                    count_1 += 1
                count_1 = 0
                info_string += net_matrix_row_seperator if count_2 != len(layer) - 1 else ''
                count_2 += 1
            count_2 = 0
            info_string += net_sub_section_seperator if count_3 != len(self.hidden_layers) - 1 else ''
            count_3 += 1
        info_string += net_section_seperator
        count_1 = 0
        count_2 = 0
        for bias in self.biases:
            for index in bias:
                info_string += f'{index}'
                info_string += net_vector_index_seperator if count_1 != len(bias) - 1 else ''
                count_1 += 1
            count_1 = 0
            info_string += net_sub_section_seperator if count_2 != len(self.biases) - 1 else ''
            count_2 += 1
        return info_string
    
    # Loads the net's attribute's from a given string and returns it's score.
    # String Format is listed above the generate_info_string method
    def load_info_from_string(self,info_string,non_linear=sigmoid):
        sections = info_string.split(net_section_seperator)
        self.name = sections[0]
        score = float(sections[1])
        self.hidden_layers = []
        self.biases = []
        self.non_linears = [non_linear]
        hidden_layer_sections = sections[2].split(net_sub_section_seperator)
        for layer in hidden_layer_sections:
            vectors = []
            for text_vector in layer.split(net_matrix_row_seperator):
                vector = []
                for index in text_vector.split(net_vector_index_seperator):
                    vector.append(float(index))
                vectors.append(vector)
            self.hidden_layers.append(np.array(vectors))
        self.non_linears *= len(self.hidden_layers)
        text_biases = sections[3].split(net_sub_section_seperator)
        for text_bias in text_biases:
            bias = []
            for index in text_bias.split(net_vector_index_seperator):
                bias.append(float(index))
            self.biases.append(np.array(bias))
        return score

In [6]:
# All functions related to saving and loading data are in this cell.

# Saves the given wrapped net population.
def save_wrapped_net_population(population,name,generation):
    try:
        file = open(f'Populations/{name}.snp','w')
        file.write(f'Generation {generation}')
        for wrapped_net in population:
            file.write(f'\n{wrapped_net[0].generate_info_string(wrapped_net[1])}')
        file.close()
    except:
        print(f'Failed to save population : {name}')
        
# Loads a net population from a given file in format (population,generation)
def load_wrapped_net_population(name):
    file = None
    try:
        file = open(f'Populations/{name}.snp','r')
    except:
        print(f'Failed to find file for population : {name}')
        return None
    if file != None:
        contents = file.read().split('\n')
        generation = int(contents[0].split(' ')[1])
        population = []
        for val in contents[1:]:
            net = generate_standard_net()
            score = net.load_info_from_string(val)
            population.append((net,score))
        return (population,generation)

In [7]:
# Generates a net_net with preset values (this is basically a way to make net generation easier).
def generate_standard_net(name="N/A"): 
    return net(input_size=42,hidden_layers=[7],name=name)
    #return net(input_size=42,hidden_layers=[20,10,7],name=name)
    #return net_net(input_size=5,hidden_layers=[3,2,1],name=name)

In [8]:
# Genetic evolution related functions are declared in this cell.

# Breeds two nets using indexed addition and the specified weight, mates child with a random net if mutating.
def breed_nets(net_1,net_2,mutation=False,non_linear=sigmoid,name="N/A"):
    hidden_layers = []
    biases = []
    non_linears = []
    for i in range(len(net_1.hidden_layers)):
        #hidden_layers.append((net_1.hidden_layers[i] * net_1_weight) + (net_2.hidden_layers[i] * (1 - net_1_weight)))
        #biases.append((net_1.biases[i] * net_1_weight) + (net_2.biases[i] * (1 - net_1_weight)))
        non_linears.append(non_linear)
        hidden_layers.append([])
        biases.append([])
        for j in range(len(net_1.hidden_layers[i])):
            hidden_layers[i].append((rand.choice([net_1,net_2])).hidden_layers[i][j])
        hidden_layers[i] = np.array(hidden_layers[i])
        for j in range(len(net_1.biases[i])):
            biases[i].append((rand.choice([net_1,net_2])).biases[i][j])
        biases[i] = np.array(biases[i])
    child = generate_standard_net(name=name)
    child.hidden_layers = hidden_layers
    child.biases = biases
    child.non_linears = non_linears
    if mutation:
        return breed_nets(child,generate_standard_net(),name=name)
    else:
        return child

# Breeds a population of ordered wrapped nets.
def breed_wrapped_net_popultation(old_population,generation):
    avg_score = 0
    old_size = len(old_population)
    for wrapped_net in old_population:
        avg_score += wrapped_net[1]
    avg_score /= len(old_population)
    new_population = []
    rand.shuffle(old_population)
    for wrapped_net in old_population:
        if wrapped_net[1] >= avg_score and len(new_population) < len(old_population) / 2:
            new_population.append(wrapped_net)
    old_population = new_population
    new_population = []
    child_num = 0
    rand.shuffle(old_population)
    while len(old_population) > 1:
        wrapped_nets = [old_population.pop(),old_population.pop()]
        score_sum = wrapped_nets[0][1] + wrapped_nets[1][1]
        child_num += 1
        child_1 = breed_nets(wrapped_nets[0][0],wrapped_nets[1][0],mutation=np.random.random(1)[0] > 0.95,name=f'Net : {child_num} (Generation : {generation})')
        child_num += 1
        child_2 = breed_nets(wrapped_nets[0][0],wrapped_nets[1][0],mutation=np.random.random(1)[0] > 0.95,name=f'Net : {child_num} (Generation : {generation})')
        new_population.append((child_1,0))
        new_population.append((child_2,0))
        if avg_score <= 0:
            if np.random.random(1)[0] > 0.5:
                new_population = add_wrapped_net(new_population,wrapped_nets[0])
            else:
                new_population = add_wrapped_net(new_population,wrapped_nets[1])
        else:
            for wrapped_net in wrapped_nets:
                if wrapped_net[1] >= avg_score:
                        new_population = add_wrapped_net(new_population,wrapped_net)
    if len(old_population) <= 3:
            child_num += 1
            new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
            child_num += 1
            new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
            child_num += 1
            new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
    while len(old_population) > 0:
        new_population = add_wrapped_net(new_population,old_population.pop())
    if len(new_population) <= 0:
        for _ in range(8):
            child_num += 1
            new_population.append((generate_standard_net_net(name=f'Net : {child_num} (Generation : {generation})'),0))
    while len(new_population) < old_size:
        child_num += 1
        new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
   # if generation % 4:
           # child_num += 1
           # new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
    if generation % 200 == 0:
        new_population = new_population[:20]
        for _ in range(25):
            child_num += 1
            new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
    return new_population
    
# Subjects a population to natural selection for the given number of generations.
def subject_popultaion_to_natural_selection(population,starting_generation,name,num_of_generations=100,verbose_crawl=False):
    for i in range(num_of_generations):
        current_generation = starting_generation + i
        print(f'Beginning Natural Selection For Generation : {current_generation} | Population Size : {len(population)}')
        new_population = []
        high_score = 0
        count = 0
        start = time.time()
        for wrapped_net in population:
            new_population.append((wrapped_net[0],0))
        for i in range(len(new_population) - 1):
            for j in range(len(new_population) - i):
                if j != 0:
                    logic_engine_1 = logic_engine_wrapper(logic_engine=population[i][0])
                    logic_engine_2 = logic_engine_wrapper(logic_engine=population[i+j][0])
                    result = play_game(logic_engines=[logic_engine_1,logic_engine_2],verbose=False,ai_delay=0)
                    if result == 1:
                        new_population[i] = (new_population[i][0],new_population[i][1]+1)
                    elif result == -1:
                        new_population[i] = (new_population[i+j][0],new_population[i+j][1]+1)
        proxy_pop = []
        for wrapped_net in new_population:
            add_wrapped_net(proxy_pop,wrapped_net)
            if wrapped_net[1] > high_score:
                high_score = wrapped_net[1]
        new_population = proxy_pop
        end = time.time()
        print(f'Generation {current_generation} High Score : {high_score} | Duration : {end - start} Secs')
        population = breed_wrapped_net_popultation(new_population,current_generation + 1)
        save_wrapped_net_population(population,name,current_generation + 1)
        print(f'Generation {current_generation + 1} Bred')

# Generates a starting population of nets. The nets are in a tuple wrapper of format (net,score)
def generate_base_wrapped_net_population(size=100):
    return [(generate_standard_net(f'Net : {i + 1} (Generation : 0)'),0) for i in range(size)]

# Adds a net wrapper to the given array in order of greatest score.
def add_wrapped_net(array,wrapped_net):
    if len(array) == 0:
        array.append(wrapped_net)
    else:
        placed = False
        for i in range(len(array)):
            if array[i][1] < wrapped_net[1] and not placed:
                array.insert(i,wrapped_net)
                placed = True
        if not placed:
            array.append(wrapped_net)
    return array

In [9]:
# Creates a new population.
population = generate_base_wrapped_net_population(10)
generation = 0

In [10]:
# Loads a population from the disk.
hold = load_wrapped_net_population('eigthTrial')
population = hold[0]
generation = hold[1]

In [11]:
subject_popultaion_to_natural_selection(population,generation,name='eigthTrial',num_of_generations=50000,verbose_crawl=False)

Beginning Natural Selection For Generation : 33364 | Population Size : 123
Generation 33364 High Score : 9 | Duration : 3.956054210662842 Secs
Generation 33365 Bred
Beginning Natural Selection For Generation : 33365 | Population Size : 123
Generation 33365 High Score : 13 | Duration : 4.410433292388916 Secs
Generation 33366 Bred
Beginning Natural Selection For Generation : 33366 | Population Size : 123
Generation 33366 High Score : 8 | Duration : 5.130659580230713 Secs
Generation 33367 Bred
Beginning Natural Selection For Generation : 33367 | Population Size : 123
Generation 33367 High Score : 7 | Duration : 5.436615228652954 Secs
Generation 33368 Bred
Beginning Natural Selection For Generation : 33368 | Population Size : 123
Generation 33368 High Score : 7 | Duration : 5.516998529434204 Secs
Generation 33369 Bred
Beginning Natural Selection For Generation : 33369 | Population Size : 123
Generation 33369 High Score : 6 | Duration : 4.395545959472656 Secs
Generation 33370 Bred
Beginning

Generation 33414 High Score : 4 | Duration : 1.1680808067321777 Secs
Generation 33415 Bred
Beginning Natural Selection For Generation : 33415 | Population Size : 48
Generation 33415 High Score : 10 | Duration : 0.7871072292327881 Secs
Generation 33416 Bred
Beginning Natural Selection For Generation : 33416 | Population Size : 48
Generation 33416 High Score : 5 | Duration : 1.1413304805755615 Secs
Generation 33417 Bred
Beginning Natural Selection For Generation : 33417 | Population Size : 51
Generation 33417 High Score : 19 | Duration : 1.1888747215270996 Secs
Generation 33418 Bred
Beginning Natural Selection For Generation : 33418 | Population Size : 51
Generation 33418 High Score : 8 | Duration : 1.0822722911834717 Secs
Generation 33419 Bred
Beginning Natural Selection For Generation : 33419 | Population Size : 51
Generation 33419 High Score : 9 | Duration : 1.0882561206817627 Secs
Generation 33420 Bred
Beginning Natural Selection For Generation : 33420 | Population Size : 51
Generati

Generation 33464 High Score : 6 | Duration : 2.3074324131011963 Secs
Generation 33465 Bred
Beginning Natural Selection For Generation : 33465 | Population Size : 87
Generation 33465 High Score : 11 | Duration : 1.6457257270812988 Secs
Generation 33466 Bred
Beginning Natural Selection For Generation : 33466 | Population Size : 87
Generation 33466 High Score : 8 | Duration : 2.346616744995117 Secs
Generation 33467 Bred
Beginning Natural Selection For Generation : 33467 | Population Size : 87
Generation 33467 High Score : 12 | Duration : 2.4735913276672363 Secs
Generation 33468 Bred
Beginning Natural Selection For Generation : 33468 | Population Size : 87
Generation 33468 High Score : 4 | Duration : 2.063323736190796 Secs
Generation 33469 Bred
Beginning Natural Selection For Generation : 33469 | Population Size : 91
Generation 33469 High Score : 5 | Duration : 1.9984049797058105 Secs
Generation 33470 Bred
Beginning Natural Selection For Generation : 33470 | Population Size : 91
Generation

Generation 33514 High Score : 9 | Duration : 3.445709228515625 Secs
Generation 33515 Bred
Beginning Natural Selection For Generation : 33515 | Population Size : 103
Generation 33515 High Score : 6 | Duration : 3.567216634750366 Secs
Generation 33516 Bred
Beginning Natural Selection For Generation : 33516 | Population Size : 103
Generation 33516 High Score : 6 | Duration : 3.308760643005371 Secs
Generation 33517 Bred
Beginning Natural Selection For Generation : 33517 | Population Size : 103
Generation 33517 High Score : 8 | Duration : 3.3707621097564697 Secs
Generation 33518 Bred
Beginning Natural Selection For Generation : 33518 | Population Size : 107
Generation 33518 High Score : 11 | Duration : 2.71506667137146 Secs
Generation 33519 Bred
Beginning Natural Selection For Generation : 33519 | Population Size : 111
Generation 33519 High Score : 25 | Duration : 2.3485474586486816 Secs
Generation 33520 Bred
Beginning Natural Selection For Generation : 33520 | Population Size : 111
Generat

Generation 33564 High Score : 12 | Duration : 5.725821256637573 Secs
Generation 33565 Bred
Beginning Natural Selection For Generation : 33565 | Population Size : 127
Generation 33565 High Score : 5 | Duration : 5.066179513931274 Secs
Generation 33566 Bred
Beginning Natural Selection For Generation : 33566 | Population Size : 127
Generation 33566 High Score : 4 | Duration : 4.400959253311157 Secs
Generation 33567 Bred
Beginning Natural Selection For Generation : 33567 | Population Size : 127
Generation 33567 High Score : 6 | Duration : 3.685774803161621 Secs
Generation 33568 Bred
Beginning Natural Selection For Generation : 33568 | Population Size : 131
Generation 33568 High Score : 5 | Duration : 5.513568878173828 Secs
Generation 33569 Bred
Beginning Natural Selection For Generation : 33569 | Population Size : 135
Generation 33569 High Score : 6 | Duration : 5.150485277175903 Secs
Generation 33570 Bred
Beginning Natural Selection For Generation : 33570 | Population Size : 135
Generatio

...

Generation 38614 High Score : 6 | Duration : 0.6720786094665527 Secs
Generation 38615 Bred
Beginning Natural Selection For Generation : 38615 | Population Size : 48
Generation 38615 High Score : 11 | Duration : 0.805504560470581 Secs
Generation 38616 Bred
Beginning Natural Selection For Generation : 38616 | Population Size : 48
Generation 38616 High Score : 7 | Duration : 0.7722682952880859 Secs
Generation 38617 Bred
Beginning Natural Selection For Generation : 38617 | Population Size : 48
Generation 38617 High Score : 6 | Duration : 0.6983184814453125 Secs
Generation 38618 Bred
Beginning Natural Selection For Generation : 38618 | Population Size : 48
Generation 38618 High Score : 5 | Duration : 0.8144264221191406 Secs
Generation 38619 Bred
Beginning Natural Selection For Generation : 38619 | Population Size : 48
Generation 38619 High Score : 13 | Duration : 0.8268256187438965 Secs
Generation 38620 Bred
Beginning Natural Selection For Generation : 38620 | Population Size : 48
Generatio

Generation 38664 High Score : 6 | Duration : 1.6442370414733887 Secs
Generation 38665 Bred
Beginning Natural Selection For Generation : 38665 | Population Size : 75
Generation 38665 High Score : 11 | Duration : 1.4989094734191895 Secs
Generation 38666 Bred
Beginning Natural Selection For Generation : 38666 | Population Size : 75
Generation 38666 High Score : 8 | Duration : 1.6804461479187012 Secs
Generation 38667 Bred
Beginning Natural Selection For Generation : 38667 | Population Size : 75
Generation 38667 High Score : 7 | Duration : 1.6581685543060303 Secs
Generation 38668 Bred
Beginning Natural Selection For Generation : 38668 | Population Size : 75
Generation 38668 High Score : 10 | Duration : 1.8535974025726318 Secs
Generation 38669 Bred
Beginning Natural Selection For Generation : 38669 | Population Size : 75
Generation 38669 High Score : 9 | Duration : 1.7320756912231445 Secs
Generation 38670 Bred
Beginning Natural Selection For Generation : 38670 | Population Size : 75
Generati

Generation 38714 High Score : 5 | Duration : 2.010737180709839 Secs
Generation 38715 Bred
Beginning Natural Selection For Generation : 38715 | Population Size : 91
Generation 38715 High Score : 13 | Duration : 2.844597816467285 Secs
Generation 38716 Bred
Beginning Natural Selection For Generation : 38716 | Population Size : 91
Generation 38716 High Score : 8 | Duration : 2.249897003173828 Secs
Generation 38717 Bred
Beginning Natural Selection For Generation : 38717 | Population Size : 91
Generation 38717 High Score : 11 | Duration : 2.337594747543335 Secs
Generation 38718 Bred
Beginning Natural Selection For Generation : 38718 | Population Size : 91
Generation 38718 High Score : 7 | Duration : 2.3828203678131104 Secs
Generation 38719 Bred
Beginning Natural Selection For Generation : 38719 | Population Size : 91
Generation 38719 High Score : 7 | Duration : 2.3292126655578613 Secs
Generation 38720 Bred
Beginning Natural Selection For Generation : 38720 | Population Size : 91
Generation 3

Generation 38764 High Score : 9 | Duration : 3.792450189590454 Secs
Generation 38765 Bred
Beginning Natural Selection For Generation : 38765 | Population Size : 107
Generation 38765 High Score : 7 | Duration : 4.447168827056885 Secs
Generation 38766 Bred
Beginning Natural Selection For Generation : 38766 | Population Size : 107


KeyboardInterrupt: 

In [73]:
population = load_wrapped_net_population('eigthTrial')[0]

In [18]:
population  = population[:30]
len(population)

30

In [32]:
logic_engine_1 = logic_engine_wrapper(logic_engine=population[3][0])
logic_engine_2 = logic_engine_wrapper(logic_engine=population[5][0])
result = play_game(logic_engines=[logic_engine_1,logic_engine_2],verbose=True,ai_delay=0)

|       |
|       |
|       |
|       |
|       |
|       |
---------
 1234567 
Thinking...
|       |
|       |
|       |
|       |
|       |
|#      |
---------
 1234567 
Thinking...
|       |
|       |
|       |
|       |
|       |
|#     @|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|       |
|#      |
|#     @|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|#      |
|#     @|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|#      |
|#    #@|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|#      |
|#@   #@|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|##     |
|#@   #@|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|##     |
|#@@  #@|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@#     |
|##     |
|#@@  #@|
---------
 1234567 
Thinking...
|       |
|       |
|@      |
|@#     |
|##     |
|#@@  #@|
---------
 1234567 


In [14]:
len(generate_standard_net().hidden_layers[0])

42